In [1]:
GPU_device_id = str(2)
model_id_save_as = 'learningcurve-cnn-full-final-reluupdate'
architecture_id = '../hyperparameter_search/hyperparameter-search-results/CNN-kfoldsfull-final-2-reluupdate_28'
model_class_id = 'CNN1D'
testing_dataset_id = '../../source-interdiction/dataset_generation/validation_dataset_full_200keV_log10time_100.npy'
training_dataset_id = '../../source-interdiction/dataset_generation/training_dataset_full_200keV_log10time_10000.npy'
difficulty_setting = 'full'

train_sizes = [10000, 50, 100, 500, 1000, 5000, 15000, 20000, ]
earlystop_patience = 10
num_epochs = 2000

In [2]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_device_id


from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from numpy.random import seed
seed(5)
from tensorflow import set_random_seed
set_random_seed(5)

#### Import model, training function 

In [3]:
from annsa.model_classes import build_cnn_model, compile_model, f1
from annsa.load_dataset import load_easy, load_full, dataset_to_spectrakeys
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Training Data Construction

In [8]:
training_dataset = np.load(training_dataset_id)
training_spectra, training_keys = dataset_to_spectrakeys(training_dataset)

## Load testing dataset

In [9]:
testing_dataset = np.load(testing_dataset_id)
testing_spectra, testing_keys = dataset_to_spectrakeys(testing_dataset)

## Load Model

In [4]:
model_features = load_features(architecture_id)
model_features.output_function = tf.nn.softmax

In [5]:
model_features.cnn_kernels = model_features.cnn_kernel
model_features.pool_sizes = model_features.pool_size
model_features.loss = tf.keras.losses.categorical_crossentropy
model_features.optimizer = tf.keras.optimizers.Adam
model_features.metrics = [f1]
model_features.dropout_rate = model_features.dropout_probability
model_features.output_function = tf.nn.softmax
model_features.input_dim = 1024

In [ ]:
model = compile_model(
            build_cnn_model,
            model_features)

## Train network

# Scale input data

In [15]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)
testing_spectra_scaled = model_features.scaler.transform(testing_spectra)

In [16]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1',
    patience=earlystop_patience,
    mode='max',
    min_delta=0.01,
    restore_best_weights=True)

In [17]:
mlb=LabelBinarizer()

for train_size in train_sizes:
    print('\n\nRunning through training size '+str(train_size))
    k_folds_errors = []

    sss = StratifiedShuffleSplit(n_splits=5, train_size=train_size)
    k = 0
    for train_index, _ in sss.split(training_spectra, training_keys):
        print('Running through fold '+str(k))
        training_keys_binarized = mlb.fit_transform(training_keys.reshape([training_keys.shape[0],1]))
        testing_keys_binarized = mlb.transform(testing_keys)
        model = compile_model(
            build_cnn_model,
            model_features)

        csv_logger = tf.keras.callbacks.CSVLogger('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.log')

        output = model.fit(
            x=training_spectra_scaled[train_index],
            y=training_keys_binarized[train_index],
            epochs=num_epochs,
            verbose=1,
            validation_data=(testing_spectra_scaled,
                             testing_keys_binarized),
            shuffle=True,
            callbacks=[earlystop_callback, csv_logger],
        )
        model.save('./final-models-keras/'+model_id_save_as+'_trainsize'+str(train_size)+'_fold'+str(k)+'.hdf5')
        k += 1




Running through training size 10000
Running through fold 0


/home/ubuntu/anaconda3/envs/tensorflow_p36_update/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 10000 samples, validate on 3000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/2000
10000/10000 [==============================] - 6s 632us/sample - loss: 3.3193 - f1: 0.0190 - val_loss: 2.6738 - val_f1: 0.0691
Epoch 2/2000
10000/10000 [==============================] - 2s 249us/sample - loss: 2.2701 - f1: 0.1299 - val_loss: 1.9334 - val_f1: 0.2116
Epoch 3/2000
10000/10000 [==============================] - 2s 249us/sample - loss: 1.7620 - f1: 0.3056 - val_loss: 1.5859 - val_f1: 0.3989
Epoch 4/2000
10000/10000 [==============================] - 2s 250us/sample - loss: 1.4928 - f1: 0.4555 - val_loss: 1.3830 - val_f1: 0.5273
Epoch 5/2000
10000/10000 [==============================] - 3s 250us/sample - loss: 1.3222 - f1: 0.5589 - val_loss: 1.2359 - val_f1: 0.6094
Epoch 6/2000
100

Epoch 32/2000
10000/10000 [==============================] - 3s 251us/sample - loss: 0.3896 - f1: 0.9352 - val_loss: 0.4580 - val_f1: 0.9073
Epoch 33/2000
10000/10000 [==============================] - 3s 252us/sample - loss: 0.3832 - f1: 0.9375 - val_loss: 0.4571 - val_f1: 0.9064
Epoch 34/2000
10000/10000 [==============================] - 3s 251us/sample - loss: 0.3784 - f1: 0.9368 - val_loss: 0.4539 - val_f1: 0.9059
Epoch 35/2000
10000/10000 [==============================] - 3s 251us/sample - loss: 0.3693 - f1: 0.9387 - val_loss: 0.4419 - val_f1: 0.9108
Epoch 36/2000
10000/10000 [==============================] - 3s 251us/sample - loss: 0.3585 - f1: 0.9406 - val_loss: 0.4270 - val_f1: 0.9147
Epoch 37/2000
10000/10000 [==============================] - 3s 251us/sample - loss: 0.3515 - f1: 0.9420 - val_loss: 0.4289 - val_f1: 0.9137
Epoch 38/2000
10000/10000 [==============================] - 3s 271us/sample - loss: 0.3497 - f1: 0.9412 - val_loss: 0.4258 - val_f1: 0.9117
Running throu

50/50 [==============================] - 1s 25ms/sample - loss: 3.8930 - f1: 0.0000e+00 - val_loss: 3.8831 - val_f1: 0.0000e+00
Epoch 2/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8652 - f1: 0.0000e+00 - val_loss: 3.8736 - val_f1: 0.0000e+00
Epoch 3/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8420 - f1: 0.0000e+00 - val_loss: 3.8658 - val_f1: 0.0000e+00
Epoch 4/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8291 - f1: 0.0000e+00 - val_loss: 3.8594 - val_f1: 0.0000e+00
Epoch 5/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.8044 - f1: 0.0000e+00 - val_loss: 3.8539 - val_f1: 0.0000e+00
Epoch 6/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.7962 - f1: 0.0000e+00 - val_loss: 3.8483 - val_f1: 0.0000e+00
Epoch 7/2000
50/50 [==============================] - 0s 6ms/sample - loss: 3.7838 - f1: 0.0000e+00 - val_loss: 3.8431 - val_f1: 0.0000e+00
Epoch 8/2000
50/50 [============

...

15000/15000 [==============================] - 4s 279us/sample - loss: 0.6863 - f1: 0.8540 - val_loss: 0.6794 - val_f1: 0.8493
Epoch 12/2000
15000/15000 [==============================] - 4s 279us/sample - loss: 0.6550 - f1: 0.8631 - val_loss: 0.6387 - val_f1: 0.8675
Epoch 13/2000
15000/15000 [==============================] - 4s 280us/sample - loss: 0.6211 - f1: 0.8723 - val_loss: 0.6227 - val_f1: 0.8692
Epoch 14/2000
15000/15000 [==============================] - 4s 279us/sample - loss: 0.5917 - f1: 0.8811 - val_loss: 0.5966 - val_f1: 0.8778
Epoch 15/2000
15000/15000 [==============================] - 4s 280us/sample - loss: 0.5679 - f1: 0.8869 - val_loss: 0.5737 - val_f1: 0.8828
Epoch 16/2000
15000/15000 [==============================] - 4s 279us/sample - loss: 0.5441 - f1: 0.8925 - val_loss: 0.5455 - val_f1: 0.8952
Epoch 17/2000
15000/15000 [==============================] - 4s 280us/sample - loss: 0.5257 - f1: 0.8996 - val_loss: 0.5328 - val_f1: 0.8957
Epoch 18/2000
15000/15000 [

Epoch 28/2000
15000/15000 [==============================] - 4s 281us/sample - loss: 0.3590 - f1: 0.9388 - val_loss: 0.3935 - val_f1: 0.9284
Epoch 29/2000
15000/15000 [==============================] - 4s 287us/sample - loss: 0.3490 - f1: 0.9422 - val_loss: 0.3858 - val_f1: 0.9279
Epoch 30/2000
15000/15000 [==============================] - 4s 280us/sample - loss: 0.3454 - f1: 0.9433 - val_loss: 0.3880 - val_f1: 0.9244
Epoch 31/2000
15000/15000 [==============================] - 4s 281us/sample - loss: 0.3356 - f1: 0.9440 - val_loss: 0.3695 - val_f1: 0.9361
Epoch 32/2000
15000/15000 [==============================] - 4s 280us/sample - loss: 0.3288 - f1: 0.9475 - val_loss: 0.3742 - val_f1: 0.9301
Epoch 33/2000
15000/15000 [==============================] - 4s 281us/sample - loss: 0.3226 - f1: 0.9469 - val_loss: 0.3632 - val_f1: 0.9341
Epoch 34/2000
15000/15000 [==============================] - 4s 281us/sample - loss: 0.3142 - f1: 0.9491 - val_loss: 0.3589 - val_f1: 0.9304
Epoch 35/2000

Epoch 49/2000
15000/15000 [==============================] - 4s 286us/sample - loss: 0.2046 - f1: 0.9808 - val_loss: 0.2597 - val_f1: 0.9626
Epoch 50/2000
15000/15000 [==============================] - 4s 286us/sample - loss: 0.2050 - f1: 0.9792 - val_loss: 0.2534 - val_f1: 0.9616
Epoch 51/2000
15000/15000 [==============================] - 4s 286us/sample - loss: 0.2018 - f1: 0.9810 - val_loss: 0.2706 - val_f1: 0.9576
Epoch 52/2000
15000/15000 [==============================] - 7s 497us/sample - loss: 0.1947 - f1: 0.9824 - val_loss: 0.2552 - val_f1: 0.9615


Running through training size 20000
Running through fold 0
Train on 20000 samples, validate on 3000 samples
Epoch 1/2000
20000/20000 [==============================] - 9s 444us/sample - loss: 2.7135 - f1: 0.0929 - val_loss: 1.8689 - val_f1: 0.2483
Epoch 2/2000
20000/20000 [==============================] - 6s 285us/sample - loss: 1.5650 - f1: 0.4164 - val_loss: 1.3159 - val_f1: 0.5753
Epoch 3/2000
20000/20000 [====================

Epoch 21/2000
20000/20000 [==============================] - 6s 292us/sample - loss: 0.3354 - f1: 0.9433 - val_loss: 0.3795 - val_f1: 0.9238
Epoch 22/2000
20000/20000 [==============================] - 6s 293us/sample - loss: 0.3256 - f1: 0.9454 - val_loss: 0.3875 - val_f1: 0.9174
Epoch 23/2000
20000/20000 [==============================] - 6s 293us/sample - loss: 0.3180 - f1: 0.9468 - val_loss: 0.3676 - val_f1: 0.9246
Epoch 24/2000
20000/20000 [==============================] - 9s 473us/sample - loss: 0.3088 - f1: 0.9478 - val_loss: 0.3551 - val_f1: 0.9277
Running through fold 2
Train on 20000 samples, validate on 3000 samples
Epoch 1/2000
20000/20000 [==============================] - 10s 476us/sample - loss: 2.7329 - f1: 0.0757 - val_loss: 1.8817 - val_f1: 0.2458
Epoch 2/2000
20000/20000 [==============================] - 6s 299us/sample - loss: 1.6454 - f1: 0.3520 - val_loss: 1.3844 - val_f1: 0.5140
Epoch 3/2000
20000/20000 [==============================] - 6s 295us/sample - loss:

Epoch 35/2000
20000/20000 [==============================] - 6s 299us/sample - loss: 0.2728 - f1: 0.9512 - val_loss: 0.3126 - val_f1: 0.9389
Epoch 36/2000
20000/20000 [==============================] - 6s 298us/sample - loss: 0.2689 - f1: 0.9526 - val_loss: 0.3068 - val_f1: 0.9382
Epoch 37/2000
20000/20000 [==============================] - 6s 305us/sample - loss: 0.2637 - f1: 0.9532 - val_loss: 0.3192 - val_f1: 0.9348
Epoch 38/2000
20000/20000 [==============================] - 6s 304us/sample - loss: 0.2596 - f1: 0.9533 - val_loss: 0.2921 - val_f1: 0.9423
Epoch 39/2000
20000/20000 [==============================] - 6s 300us/sample - loss: 0.2564 - f1: 0.9548 - val_loss: 0.3021 - val_f1: 0.9387
Epoch 40/2000
20000/20000 [==============================] - 6s 301us/sample - loss: 0.2511 - f1: 0.9543 - val_loss: 0.3032 - val_f1: 0.9398
Epoch 41/2000
20000/20000 [==============================] - 10s 505us/sample - loss: 0.2467 - f1: 0.9566 - val_loss: 0.2966 - val_f1: 0.9379
